<a href="https://colab.research.google.com/github/techfreakydeepak/Bike_sharing_demand_prediction/blob/main/Health_Inssurance_Cross_Sell_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# here we will import the libraries used for machine learning
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O, data manipulation 
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from pandas import set_option
plt.style.use('ggplot') # nice plots

In [ ]:
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold # for cross validation
from sklearn.model_selection import GridSearchCV # for tuning parameter
from sklearn.model_selection import RandomizedSearchCV  # Randomized search on hyper parameters.
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os

In [ ]:
# Setting option to display all the columns in dataset
pd.set_option("display.max_columns",None)
#from pandas.io.common import is _url
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [19]:
#Mounting the drive
from google.colab import drive
drive.mount("/content/drive")
     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Give the path of the folder to access all the csv data files
url='https://drive.google.com/file/d/1BDojWtEMWK615Ye_UHPrFTVTIywl_eas/view?usp=sharing' 
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

In [21]:
# Let's read the csv file
df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION=pd.read_csv(url)

Dataset First View

In [22]:
# check top five rows to get the overview of data
df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


Dataset Rows & Columns count

In [23]:
# shape tell us rows and columns
print('the shape of data is - ',df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION.shape)

the shape of data is -  (381109, 12)


 * By this we can say that in our dataset 381109 rows and 12 columns

Dataset Information

In [24]:
# Let's get some information about data
df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


 Understanding Your Variables

Variables Description

In [ ]:
# columns Name
df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION.columns

Variables Description

The dataset contains weather information ('id', 'Gender', 'Age', 

'Driving_License', 'Region_Code','Previously_Insured', 'Vehicle_Age', 

'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel', 'Vintage', 

'Response')

Attribute Information:

* id : Unique ID for the customer

* Gender : Gender of the customer

* Age : Age of the customer

* Driving_License: 0 - Customer does not have DL, 1 - Customer already has DL

* Region_Code : Unique code for the region of the customer

* Previously_Insured : 1 - Customer already has Vehicle Insurance, 0 - Customer doesn't have Vehicle Insurance

* Vehicle_Age : Age of the Vehicle

* Vehicle_Damage :1 - Customer got his/her vehicle damaged in the past. 0 - Customer didn't get his/her vehicle damaged in the past.

* Annual_Premium : The amount customer needs to pay as premium in the year

* PolicySalesChannel : Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.

* Vintage : Number of Days, Customer has been associated with the company

* Response : 1 : Customer is interested, 0 : Customer is not interested

In [25]:
 # check description of dataset
df_HEALTH_INSURANCE_CROSS_SELL_PREDICTION.describe(include='all')

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109,381109.000000,381109.000000,381109.000000,381109.000000,381109,381109,381109.000000,381109.000000,381109.000000,381109.000000
unique,NaN,2,NaN,NaN,NaN,NaN,3,2,NaN,NaN,NaN,NaN
top,NaN,Male,NaN,NaN,NaN,NaN,1-2 Year,Yes,NaN,NaN,NaN,NaN
freq,NaN,206089,NaN,NaN,NaN,NaN,200316,192413,NaN,NaN,NaN,NaN
mean,190555.000000,NaN,38.822584,0.997869,26.388807,0.458210,NaN,NaN,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,NaN,15.511611,0.046110,13.229888,0.498251,NaN,NaN,17213.155057,54.203995,83.671304,0.327936
min,1.000000,NaN,20.000000,0.000000,0.000000,0.000000,NaN,NaN,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,NaN,25.000000,1.000000,15.000000,0.000000,NaN,NaN,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,NaN,36.000000,1.000000,28.000000,0.000000,NaN,NaN,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,NaN,49.000000,1.000000,35.000000,1.000000,NaN,NaN,39400.000000,152.000000,227.000000,0.000000
